## Building Advanced RAG Q&A Project With Multiple Data Sources With Langchain

In [2]:
from langchain_community.tools import WikipediaQueryRun 
from langchain_community.utilities import WikipediaAPIWrapper

In [3]:
import os
# 设置http的代理和不代理的地址
os.environ['HTTP_PROXY'] = "http://127.0.0.1:7890"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:7890"
os.environ['NO_PROXY'] = "127.0.0.1,localhost" #ollama的本地服务地址


## API wrapper
- this is basically use to query answer from wikipedia

In [4]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [5]:
wiki.name

'wikipedia'

In [6]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [7]:
from dotenv import load_dotenv
import os
load_dotenv()
os.environ["OPENAI_API_BASE"] = os.getenv("OPENAI_API_BASE")
os.environ['OPENAI_API_KEY']=os.getenv('OPENAI_API_KEY')

True

In [8]:
loader=WebBaseLoader('https://docs.smith.langchain.com/')
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)

In [9]:
# 临时增加代理
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

# 临时增加代理
# os.environ['HTTP_PROXY'] = "http://127.0.0.1:7890"
# os.environ['HTTPS_PROXY'] = "http://127.0.0.1:7890"

model_name = "BAAI/bge-large-zh-v1.5"
model_kwargs = {'device': 'cpu', "trust_remote_code":True}
encode_kwargs = {'normalize_embeddings': True}
hf_emb_model = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)
# 去除增加代理
# os.environ.pop('HTTP_PROXY')
# os.environ.pop('HTTPS_PROXY')

## creating vector DB
vectordb=FAISS.from_documents(documents, hf_emb_model)
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x73e652399650>, search_kwargs={})

In [10]:
from langchain_core.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith_search","Search for information about Langsmith. For any questions about LangSmith, youmust use this tool")

In [11]:
retriever_tool.name

'langsmith_search'

## Arxiv Tool

In [12]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun


In [13]:
arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(arxiv_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

## Combine All Tools

In [14]:
tools=[wiki,arxiv,retriever_tool]

In [15]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/home/chester/gitlab.chesterwang.com/ai_demo_4_interview/RAG/RAG-Level-01/P01-RAG-Projects/.venv/lib/python3.11/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=3, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=4000)),
 Tool(name='langsmith_search', description='Search for information about Langsmith. For any questions about LangSmith, youmust use this tool', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x73e65186d4e0>, retriever=VectorStoreRetriever(tags

## Agents
- The core idea of agents is to use a language model to choose a sequence of actions to take. In chains, a sequence of actions is hardcoded (in code). In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.
[Learn more](https://python.langchain.com/v0.1/docs/modules/agents/)

In [16]:
from langchain_openai import ChatOpenAI
# llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
# llm = ChatOpenAI(model="kimi-k2-turbo-preview", temperature=0) # 会遇到httpcore的问题
# from langchain_community.llms import Ollama,ChatOllama
from langchain_ollama import ChatOllama
# llm = Ollama(model="deepseek-r1:1.5b",
#     base_url="http://localhost:11434"  # 明确指定 Ollama 服务地址
# )
llm = ChatOllama(model="qwen3:0.6b",
    base_url="http://localhost:11434"  # 明确指定 Ollama 服务地址
)

In [17]:
llm.invoke("你好")

AIMessage(content='你好！有什么可以帮到你的吗？😊', additional_kwargs={}, response_metadata={'model': 'qwen3:0.6b', 'created_at': '2025-11-12T02:48:58.058410561Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3522255749, 'load_duration': 305408009, 'prompt_eval_count': 11, 'prompt_eval_duration': 22854780, 'eval_count': 116, 'eval_duration': 3094865687, 'model_name': 'qwen3:0.6b', 'model_provider': 'ollama'}, id='lc_run--080827ba-527d-4f50-8d46-aa79f3bbcc7a-0', usage_metadata={'input_tokens': 11, 'output_tokens': 116, 'total_tokens': 127})

In [18]:
from langchain_core.prompts import load_prompt
# prompt = load_prompt("hwchase17/openai-functions-agent")
# prompt.messages
from langsmith import Client
client = Client(api_key=os.getenv("LANGSMITH_API_KEY"))
prompt = client.pull_prompt("hwchase17/openai-functions-agent", include_model=False)
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [19]:
from langchain_classic.agents import create_openai_tools_agent
#这里的tools参数主要提供工具的描述信息(因为内部要对)
agent=create_openai_tools_agent(llm,tools,prompt)

## Agent Executor

In [20]:
from langchain_classic.agents import AgentExecutor
#这里的tools参数主要具体的执行函数
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [25]:
response=agent_executor.invoke({"input":"What's the paper 1605.08386 about?"})
# 这里使用deepseek-r1:1.5b 的结果是总是不会调用相关工具。



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1605.08386'}`


Published: 2016-05-27
Title: Heat-bath random walks with Markov bases
Authors: Caprice Stanley, Tobias Windisch
Summary: Graphs on lattice points are studied whose edges come from a finite set of allowed moves of arbitrary length. We show that the diameter of these graphs on fibers of a fixed integer matrix can be bounded from above by a constant. We then study the mixing behaviour of heat-bath random walks on these graphs. We also state explicit conditions on the set of moves so that the heat-bath random walk, a generalization of the Glauber dynamics, is an expander in fixed dimension.The paper titled **"Heat-bath random walks with Markov bases"** was published on **2016-05-27**.  
**Authors**: Caprice Stanley, Tobias Windisch.  
**Summary**:  
Graphs on lattice points with edges derived from finite sets of allowed moves of arbitrary length are studied. The paper shows that the **diameter** of 

In [26]:
print(response)

{'input': "What's the paper 1605.08386 about?", 'output': 'The paper titled **"Heat-bath random walks with Markov bases"** was published on **2016-05-27**.  \n**Authors**: Caprice Stanley, Tobias Windisch.  \n**Summary**:  \nGraphs on lattice points with edges derived from finite sets of allowed moves of arbitrary length are studied. The paper shows that the **diameter** of these graphs on fibers of a fixed integer matrix can be bounded from above by a constant. It also investigates the **mixing behavior** of heat-bath random walks on these graphs. Additionally, the paper states that **explicit conditions** on the set of moves ensure the heat-bath random walk (a generalization of Glauber dynamics) behaves as an **expander** in fixed dimension.'}


In [23]:
response=agent_executor.invoke({"input":"introduce the GPU to me?"})
# 这里使用deepseek-r1:1.5b 的结果是总是不会调用相关工具。



> Entering new AgentExecutor chain...
A GPU (Graphics Processing Unit) is a parallel processor designed to handle graphics and compute tasks efficiently. It's used in computers for tasks like gaming, video editing, and 3D modeling. Unlike CPUs, which are slower, GPUs can process thousands of threads at once, making them much faster for tasks that require visual processing. Examples include NVIDIA's GPUs and AMD's GPUs. They’re optimized for parallel computations, making them essential in modern computing.

> Finished chain.


In [41]:
print(response)

{'input': 'introduce the GPU to me?', 'output': "A GPU (Graphics Processing Unit) is a parallel processor used in computers to accelerate data processing tasks. It's responsible for rendering graphics, such as in games, video editing, or machine learning, by handling complex calculations that a CPU would take time to process. Unlike a CPU, which is slower, GPUs can handle multiple tasks simultaneously, making them ideal for applications that require significant computational power."}
